# ARI2129 Assignment - Part 1

Importing necessary libraries 

In [73]:
import cv2 as cv
import numpy as np

### Stage 1

This stage requires to blend an image from an object, taken from one image and adding it onto another image.

img_s1, contains an image with a single object, shampoo bottle.

img_s2, contains an image with two objetcs, shampoo bottle and deodarant.

img_mask, contains mask of shampoo bottle.

bg_book, backgorund with a book.

bg_per, background with computer peripherals.

bg_statue, background with religious statue.

bg_vase, background with vase.

In [74]:
#Images
img_s1 = cv.imread("images/Part1/objects/single_1_obj.jpeg")
img_s2 = cv.imread("images/Part1/objects/mult_2_obj.jpeg")
#Mask
img_mask = cv.imread("images/Part1/mask/wash_mask.png")
#Background
bg_book = cv.imread("images/Part1/backgrounds/book.jpeg")
bg_per = cv.imread("images/Part1/backgrounds/peripherals.jpeg")
bg_statue = cv.imread("images/Part1/backgrounds/statue.jpeg")
bg_vase = cv.imread("images/Part1/backgrounds/vase.jpeg")


Testing

In [3]:
cv.imshow("test", img_s1)
cv.waitKey(0)
cv.destroyAllWindows()

-1

Question 1

In [75]:
def ExtractObject(s2, objectMask):
    
    img_extract = s2*objectMask
    #Invert colours, since it is multiplied with white pixels of mask
    img_extract= cv.bitwise_not(img_extract)
    
    return img_extract

Testing

In [23]:
extractedObject = ExtractObject(img_s2, img_mask)
cv.imshow("test", extractedObject)
cv.waitKey(0)
cv.destroyAllWindows()

Question 2

In [76]:
def ApplyFilter(extractedObject, filterIndex):

    #Apply no filter
    if(filterIndex==0):
    
        filteredExObject = extractedObject
    
    #Apply Opening filter
    elif(filterIndex==1):
         
        kernel = np.ones((3,3), np.uint8)
        
        filteredExObject = cv.morphologyEx(extractedObject, cv.MORPH_OPEN, kernel)
    
    #Apply Gaussian blur
    elif(filterIndex==2):
        
        kernel = np.ones((5,5),np.float32)/40
        filteredExObject = cv.filter2D(extractedObject,-1,kernel)
        
    #Apply Histogram Equalisation filter
    elif(filterIndex==3):
        
        img_yuv = cv.cvtColor(extractedObject, cv.COLOR_BGR2YUV)
        img_yuv[:,:,0] = cv.equalizeHist(img_yuv[:,:,0])
        filteredExObject = cv.cvtColor(img_yuv, cv.COLOR_YUV2BGR)
        
    else:
        
        print("Out of range, please use between 1-3.")
        
    return filteredExObject

Testing

In [60]:
filterImage = ApplyFilter(extractedObject, 1)

cv.imshow("test1", filterImage)
cv.waitKey(0)
cv.destroyAllWindows()

Question 3

In [77]:
def ObjectBlender(s1, filteredExObject):
    
    alpha = 0.70
    beta = 1.0 - alpha
    blendingResult = cv.addWeighted(s1, alpha, filteredExObject, beta, -60)
    
    return blendingResult

Testing

In [61]:
blendingResult = ObjectBlender(img_s1, filterImage)
cv.imshow('Blended', blendingResult)
cv.waitKey(0)
cv.destroyAllWindows()

Question 4

In [78]:
def CompareResult(blendingResult, s2, metric):
    
    #Mean Squared Error
    if(metric==1):
        
        error = np.sum((blendingResult.astype("float") - s2.astype("float"))** 2)
        error /= float(blendingResult.shape[0] * blendingResult.shape[1])
    
    #Sum of Squared Distance Error
    elif(metric==2):
        
         error = np.sum((blendingResult-s2)**2)
    else:
        
        print("Out of range, please use between 1-2.")
        
    return error

Testing

In [201]:
print(CompareResult(blendingResult, img_s2, 1))
print(CompareResult(blendingResult, img_s2, 2))

835.7566786024306
144981515


Now we will blend the object in the image using all the different types of filters (no filter,Opening filter,Gaussian blur,Histogram Equalisation) and comapre the originale image with ours using mean squared error and sum of squared distance error.

Extracting object with its mask

In [79]:
extractedObject = ExtractObject(img_s2, img_mask)

Applying filter to extracted object

In [80]:
filterImage0 = ApplyFilter(extractedObject, 0)
filterImage1 = ApplyFilter(extractedObject, 1)
filterImage2 = ApplyFilter(extractedObject, 2)
filterImage3 = ApplyFilter(extractedObject, 3)

Blending the object into the image

In [81]:
blending0 = ObjectBlender(img_s1, filterImage0)
blending1 = ObjectBlender(img_s1, filterImage1)
blending2 = ObjectBlender(img_s1, filterImage2)
blending3 = ObjectBlender(img_s1, filterImage3)

Calculating the error value

In [82]:
print("No Filter\n")
print("Mean Squared Error:", CompareResult(blending0 , img_s2, 1))
print("Sum of Squared Distance Error:", CompareResult(blending0 , img_s2, 2))
print('---------------------------')
print("Opening filter\n")
print("Mean Squared Error:", CompareResult(blending1 , img_s2, 1))
print("Sum of Squared Distance Error:", CompareResult(blending1 , img_s2, 2))
print('---------------------------')
print("Gaussian blur\n")
print("Mean Squared Error:", CompareResult(blending2 , img_s2, 1))
print("Sum of Squared Distance Error:", CompareResult(blending2 , img_s2, 2))
print('---------------------------')
print("Histogram Equalisation filter\n")
print("Mean Squared Error:", CompareResult(blending3 , img_s2, 1))
print("Sum of Squared Distance Error:", CompareResult(blending3 , img_s2, 2))

No Filter

Mean Squared Error: 938.0311490885416
Sum of Squared Distance Error: 144692003
---------------------------
Opening filter

Mean Squared Error: 946.1434233940972
Sum of Squared Distance Error: 144695891
---------------------------
Gaussian blur

Mean Squared Error: 3498.710486111111
Sum of Squared Distance Error: 319639232
---------------------------
Histogram Equalisation filter

Mean Squared Error: 1295.0648752170139
Sum of Squared Distance Error: 144928653


### Stage 2

This stage requires removing the green screen from the image and replacing it with 4 different backgrounds from the COTS dataset.

Question 1

Testing values for upper and lower bound

In [83]:
lower_bound = np.array([0, 25, 0])
upper_bound = np.array([100, 120, 100])
    
mask = cv.inRange(img_s1,lower_bound, upper_bound)

cv.imshow('mask', mask)
cv.waitKey(0)
cv.destroyAllWindows()

In [84]:
def RemoveGreen(img):
    
    #Defining upper and lower bound of green colour
    lower_bound = np.array([0, 25, 0])
    upper_bound = np.array([100, 120, 100])
    
    mask = cv.inRange(img_s1,lower_bound, upper_bound)
    
    imgNoBg = np.copy(img)
    imgNoBg[mask != 0] = [0, 0, 0]
    
    cv.imshow('imgNoBg', imgNoBg)
    cv.waitKey(0)
    cv.destroyAllWindows()
    
    return mask, imgNoBg

Testing

In [66]:
imgNoBg=RemoveGreen(img_s1)

Question 2

In [85]:
def NewBackground(imgNoBg, newBackGround):
    
    mask, image = RemoveGreen(imgNoBg)
    
    crop_background = cv.resize(newBackGround,(imgNoBg.shape[1], imgNoBg.shape[0]))
    
    crop_background[mask == 0] = [0, 0, 0]
    
    back = np.array(image + crop_background)
    
    cv.imshow('f', back)
    cv.waitKey(0)
    cv.destroyAllWindows()
    
    return back

Testing

In [69]:
final = NewBackground(img_s1, bg_per)
cv.imshow('final', final)
cv.waitKey(0)
cv.destroyAllWindows()

Adding the shampoo bottle to four different backgrounds

In [86]:
imgbg1=NewBackground(img_s1, bg_book)
imgbg2=NewBackground(img_s1, bg_per)
imgbg3=NewBackground(img_s1, bg_statue)
imgbg4=NewBackground(img_s1, bg_vase)